<a href="https://colab.research.google.com/github/StevenPeutz/Masterthesis-Disinformation-NLP/blob/master/PCA_GloVe%26FastText/Experimental_FastTextUtilReduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dimension Reduction FastText (Alternative methods & Experimentation)

Fasttext (reduced to 50 dim)


In [ ]:
# !pip install fasttext #==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=4396453 sha256=01d3b3435b360d2f29fdf5f426aa5731072cf5eaecee52957113c1c0383d9267
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


# Imports

In [ ]:
import fasttext
#import fasttext.util

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
from urllib.request import urlopen

In [ ]:
#from google.colab import drive
from google.colab import drive

# Mounting

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# prepping columns etc

In [ ]:
pathTESTcsv = '/content/drive/MyDrive/MYDATA/full_TEST01_NX.csv' #be mindful, this is Test-INC (as the INC is the one with the translation columns)
pathTRAINcsv = '/content/drive/MyDrive/MYDATA/df_full2_TRAIN01split.csv' #NTS: this trainset is not capped at 2800 chars.. (the testset is..)

df_test = pd.read_csv(pathTESTcsv)
df_train = pd.read_csv(pathTRAINcsv)

In [ ]:
df_train = df_train.sample(1000)

In [ ]:
X_train = df_train['text']
y_train = df_train['label']
X_testN0 = df_test['text']
X_testN1 = df_test['text_N1']
X_testN2 = df_test['text_N2']
X_testN3 = df_test['text_N3']
y_testALL = df_test['label']

# tokenizing, sequencing and padding..


In [ ]:
word_tokenizer = Tokenizer(num_words = 1000)
word_tokenizer.fit_on_texts(X_train)

word_index = word_tokenizer.word_index
vocab_length = len(word_index) + 1 #cant be unique words recheck this. Probably need cleaning of data first so that e.g. @CNN is not counted as unique word
# https://github.com/keras-team/keras/issues/7551
#vocab_length -> 267981 / 231677

In [ ]:
#convert series to list (prior to this 2.7GB used)
X_train_sequences = word_tokenizer.texts_to_sequences(X_train)
# ---
X_testN0_sequences = word_tokenizer.texts_to_sequences(X_testN0)
X_testN1_sequences = word_tokenizer.texts_to_sequences(X_testN1)
X_testN2_sequences = word_tokenizer.texts_to_sequences(X_testN2)
X_testN3_sequences = word_tokenizer.texts_to_sequences(X_testN3)

In [ ]:
# Output the results to inspect what is happening
print("Word index for example word 'home' :\n", word_index.get('home')) 
print("\nTraining sequence of first doc:\n", X_train_sequences[0]) #first doc, '106' is index of first word of this doc
print("\nX_train_sequences data type:", type(X_train_sequences))

Word index for example word 'home' :
 286

Training sequence of first doc:
 [474, 4, 49, 442, 271, 10, 8, 258, 2, 51, 8, 967, 53, 5, 833, 51, 6, 4, 2, 1, 968, 3, 865, 721, 36, 11, 97, 22, 249, 146, 834, 69, 866, 5, 51, 411, 69, 596, 2, 866, 1, 5, 644, 12, 521, 2, 16, 721, 30, 24, 691, 505, 77, 197, 45, 16, 1, 97, 15, 1, 188, 442, 188, 531, 442, 38, 21, 753, 69, 1, 506, 114, 3, 11, 708, 6, 195, 721, 6, 30, 737, 432, 738, 24, 1, 153, 2, 200, 2, 497, 6, 40, 286, 777, 33, 25, 22, 355, 2, 967, 189, 18, 155, 5, 111, 5, 1, 3, 31, 362, 139, 9, 7, 362, 38, 79, 21, 5, 1, 109, 38, 21, 941, 2, 721, 8, 1, 812, 3, 4, 362, 7, 34, 11, 813, 334, 1, 1, 271, 103, 86, 196, 1, 223, 969, 1, 402, 867, 43, 22, 15, 12, 38, 970, 355, 87, 97, 8, 531, 5, 721, 2, 134, 40, 87, 1, 866, 335, 3, 1, 269, 223, 98, 22, 64, 721, 2, 54, 19, 507, 2, 118, 475, 1, 87, 1, 3, 835, 778, 22, 464, 1, 10, 79, 545, 11, 23, 12, 105, 991, 59, 464, 384, 37, 21, 597, 33, 12, 11, 23, 1, 212, 10, 79, 1, 64, 914, 230, 55, 69, 1, 259, 506, 

In [ ]:
# Get max training sequence length
maxlenCalc = max([len(x) for x in X_train_sequences])
print(maxlenCalc) # = 814

416


In [ ]:
#set some restrictive limits to make it fast for now
#max_features = 500
max_len = 180 #replace with: maxlenCalc = max([len(x) for x in X_train_sequences])
              #or replace with desired length of dim divided by 50 (length of vectors per word)
padding_type='post'
truncation_type='post'

In [ ]:
#padding or truncating the lists (sequences) to make equal length. Now Numpy Arrays
# replace with maxlenCalc (887), but keep at 300 for now for speed..
X_train_SeqPad = pad_sequences(X_train_sequences, maxlen=max_len, padding=padding_type, truncating=truncation_type)
# ---
X_testN0_SeqPad = pad_sequences(X_testN0_sequences, maxlen=max_len, padding=padding_type, truncating=truncation_type)
X_testN1_SeqPad = pad_sequences(X_testN1_sequences, maxlen=max_len, padding=padding_type, truncating=truncation_type)
X_testN2_SeqPad = pad_sequences(X_testN2_sequences, maxlen=max_len, padding=padding_type, truncating=truncation_type)
X_testN3_SeqPad = pad_sequences(X_testN3_sequences, maxlen=max_len, padding=padding_type, truncating=truncation_type)


In [ ]:
# Output the results to inspect what is happening
print("\nPadded training sequences of first doc:\n", X_train_SeqPad[0])
print("\nX_train_SeqPad data type:", type(X_train_SeqPad))
print("\nX_train_SeqPad shape:", X_train_SeqPad.shape) #rows (67625) by maxlen (100)


Padded training sequences of first doc:
 [474   4  49 442 271  10   8 258   2  51   8 967  53   5 833  51   6   4
   2   1 968   3 865 721  36  11  97  22 249 146 834  69 866   5  51 411
  69 596   2 866   1   5 644  12 521   2  16 721  30  24 691 505  77 197
  45  16   1  97  15   1 188 442 188 531 442  38  21 753  69   1 506 114
   3  11 708   6 195 721   6  30 737 432 738  24   1 153   2 200   2 497
   6  40 286 777  33  25  22 355   2 967 189  18 155   5 111   5   1   3
  31 362 139   9   7 362  38  79  21   5   1 109  38  21 941   2 721   8
   1 812   3   4 362   7  34  11 813 334   1   1 271 103  86 196   1 223
 969   1 402 867  43  22  15  12  38 970 355  87  97   8 531   5 721   2
 134  40  87   1 866 335   3   1 269 223  98  22  64 721   2  54  19 507]

X_train_SeqPad data type: <class 'numpy.ndarray'>

X_train_SeqPad shape: (1000, 180)


# FastText > Utils Reduce Method (worked)

In [ ]:
#fasttext_path = '/content/drive/MyDrive/MYDATA/Embeddings_PreTrained/FastText/cc.en.300.bin'

In [ ]:
#!pip install fasttext==0.6.0

In [ ]:
#ft = fasttext.load_model(urlopen("https://drive.google.com/file/d/1f294foDnmwJs0t008opDikTJoWEHtNzr/view?usp=sharing"))
#ft = fasttext.load_model("https://drive.google.com/file/d/1f294foDnmwJs0t008opDikTJoWEHtNzr/view?usp=sharing")

ValueError: ignored

In [ ]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('/content/drive/MyDrive/MYDATA/Embeddings_PreTrained/FastText/cc.en.300.bin')

In [ ]:
fasttext.util.reduce_model(ft, 50)

In [ ]:
print(ft.get_dimension())

50


In [ ]:
ft.save_model("FastText_UtilReduced50dim.bin")

In [ ]:
ft_ft = fasttext.load_model("FastText_UtilReduced50dim.bin")

In [ ]:
ft_ft.save_model('FastText_UtilReduced50dim.txt')

In [ ]:
!cp FastText_UtilReduced50dim.txt /content/drive/MyDrive/MYDATA/Embeddings_PreTrained/FastText/

# Embedding Matrix Original

In [ ]:
#EMBEDDING_INDEX
#Creating the GloVe Embedding Index (aka embedding dictionary)
#This does not use anything else than the GloVe data (words and their vectors)

embedding_dim = 50 #based on glovefile..  ('...50d.txt')
embeddings_index = {} # == 'embeddings_dictionary' in https://www.kaggle.com/code/andreshg/nlp-glove-bert-tf-idf-lstm-explained#5.-Vectorization
glovefile = open('/content/drive/MyDrive/MYDATA/glove.6B/glove.6B.50d.txt')
for line in glovefile:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
glovefile.close()

print(f'Found {len(embeddings_index)} word vectors.')
#note to self: the dim number (50) does not influence the total word vector count (i.e. 400k for either 50 or 100 dims..)

In [ ]:
#EMBEDDING_MATRIX
#this is where we need to grab each word from the padded sequences and match them with the embeddings index to create the embedding_matrix
embedding_matrix = np.zeros((vocab_length, embedding_dim))

for word, embedding in word_tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[embedding] = embedding_vector
        
#embedding_matrix #numpy array

In [ ]:
#taking the padded sequences (word indexes per row), and replacing each index with an array of 50 vectors 
#(taken from the embedding matrix because that is where the indexes match between train and glove)

#this (in this small form) takes up about 2.7gb RAM..

listylist = []
for x in X_train_SeqPad:
  y = embedding_matrix[x]
  listylist.append(y)

# the test sets:
listylist_testN0 = []
for x in X_testN0_SeqPad:
  y = embedding_matrix[x]
  listylist_testN0.append(y)

listylist_testN1 = []
for x in X_testN1_SeqPad:
  y = embedding_matrix[x]
  listylist_testN1.append(y)

listylist_testN2 = []
for x in X_testN2_SeqPad:
  y = embedding_matrix[x]
  listylist_testN2.append(y)

listylist_testN3 = []
for x in X_testN3_SeqPad:
  y = embedding_matrix[x]
  listylist_testN3.append(y)

#prior to this block: 3.2 gb
#after: 5.9

In [ ]:
#nested list to np array (np array is better for use later on)
#another 2 gb of ram
nparraylist_train = np.array(listylist)
# ---
nparraylist_testN0 = np.array(listylist_testN0)
nparraylist_testN1 = np.array(listylist_testN1)
nparraylist_testN2 = np.array(listylist_testN2)
nparraylist_testN3 = np.array(listylist_testN3)
#prior to this block: 5.9 gb
#after: 8.2

In [ ]:
#reshape parameters (3dim array to 2dim array)

lenDim1_train = len(nparraylist_train) #67625
lenDim1_testN0 = len(nparraylist_testN0) #684

lenDim2_train = len(nparraylist_train[0]) * len(nparraylist_train[0][0]) #5000
lenDim2_testN0 = len(nparraylist_testN0[0]) * len(nparraylist_testN0[0][0]) #5000

In [ ]:
#however these are 3 dims, and need to be 2.. (1 rows, 2nd the array of vectors..)
# this can be done with reshape (see below)

X_train_GloVe = np.reshape(
               nparraylist_train,     # the array to be reshaped
               (lenDim1_train, lenDim2_train)  # dimensions of the new array
              )

# --- (change to for loop to get these 4 sets reshaped in one block)
X_testN0_GloVe = np.reshape(
               nparraylist_testN0,     # the array to be reshaped
               (lenDim1_testN0, lenDim2_testN0)  # dims of test
              )

X_testN1_GloVe = np.reshape(
               nparraylist_testN1,     # the array to be reshaped
               (lenDim1_testN0, lenDim2_testN0)  # no diff between N0 and N1 ..
              )

X_testN2_GloVe = np.reshape(
               nparraylist_testN2,     # the array to be reshaped
               (lenDim1_testN0, lenDim2_testN0)  
              )

X_testN3_GloVe = np.reshape(
               nparraylist_testN3,     # the array to be reshaped
               (lenDim1_testN0, lenDim2_testN0) 
              )
#prior to this block: 8.2 gb
#after: 8.2

In [ ]:
df_train, embedding_matrix, X_train_SeqPad, X_testN0_SeqPad, X_testN1_SeqPad, X_testN2_SeqPad, X_testN3_SeqPad, embeddings_index, df_test, word_index, X_train_sequences, X_testN0_sequences, X_testN1_sequences, X_testN2_sequences, X_testN3_sequences = [], [], [], [], [], [], [], [], [], [], [], [], [], [], []
#del df_train, X_train, embedding_matrix, X_train_SeqPad, X_testN0_SeqPad, X_testN1_SeqPad, X_testN2_SeqPad, X_testN3_SeqPad, embeddings_index, df_test, word_index, X_train_sequences, X_testN0_sequences, X_testN1_sequences, X_testN2_sequences, X_testN3_sequences

# Through Gensim (failed)

In [ ]:
# https://datascience.stackexchange.com/questions/20071/how-do-i-load-fasttext-pretrained-model-with-gensim

In [ ]:
#fasttext_path = '/content/drive/MyDrive/MYDATA/Embeddings_PreTrained/FastText/cc.en.300.bin'

In [ ]:
!pip install gensim==3.8.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from gensim.models.fasttext import load_facebook_model

In [ ]:
fasttext = load_facebook_model(fasttext_path)
#model = KeyedVectors.load_facebook_vectors(fasttext_path)
>>> import fasttext.util
>>> fasttext.util.download_model('en', if_exists='ignore')  # English
>>> ft = fasttext.load_model('cc.en.300.bin')

In [ ]:
model = FastText()
model.load_model('/path/to/model.bin')

# My own method

In [ ]:
#EMBEDDING_INDEX
#Creating the GloVe Embedding Index (aka embedding dictionary)
#This does not use anything else than the GloVe data (words and their vectors)

embedding_dim = 100 #based on glovefile..  ('...50d.txt')
embeddings_index = {} # == 'embeddings_dictionary' in https://www.kaggle.com/code/andreshg/nlp-glove-bert-tf-idf-lstm-explained#5.-Vectorization
wiki_ft_file = open('/content/drive/MyDrive/MYDATA/Embeddings_PreTrained/word2vec/model.txt')
for line in wiki_ft_file:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
wiki_ft_file.close()

print(f'Found {len(embeddings_index)} word vectors.')
#note to self: the dim number (50) does not influence the total word vector count (i.e. 400k for either 50 or 100 dims..)

UnicodeDecodeError: ignored

In [ ]:
#similar to mine, but more elegant:
#https://datascience.stackexchange.com/questions/20071/how-do-i-load-fasttext-pretrained-model-with-gensim
"""
from gensim.models import FastText
def load_fasttext():
        print('loading word embeddings...')
        embeddings_index = {}
        f = open('../input/fasttext/wiki.simple.vec',encoding='utf-8')
        for line in tqdm(f):
        values = line.strip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        f.close()
        print('found %s word vectors' % len(embeddings_index))
    
        return embeddings_index

embeddings_index=load_fastext()
"""

In [ ]:
#EMBEDDING_MATRIX
#this is where we need to grab each word from the padded sequences and match them with the embeddings index to create the embedding_matrix
embedding_matrix = np.zeros((vocab_length, embedding_dim))

for word, embedding in word_tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[embedding] = embedding_vector
        
#embedding_matrix #numpy array

In [ ]:
#taking the padded sequences (word indexes per row), and replacing each index with an array of 50 vectors 
#(taken from the embedding matrix because that is where the indexes match between train and glove)

#this (in this small form) takes up about 2.7gb RAM..

listylist = []
for x in X_train_SeqPad:
  y = embedding_matrix[x]
  listylist.append(y)

# the test sets:
listylist_testN0 = []
for x in X_testN0_SeqPad:
  y = embedding_matrix[x]
  listylist_testN0.append(y)

listylist_testN1 = []
for x in X_testN1_SeqPad:
  y = embedding_matrix[x]
  listylist_testN1.append(y)

listylist_testN2 = []
for x in X_testN2_SeqPad:
  y = embedding_matrix[x]
  listylist_testN2.append(y)

listylist_testN3 = []
for x in X_testN3_SeqPad:
  y = embedding_matrix[x]
  listylist_testN3.append(y)

#prior to this block: 3.2 gb
#after: 5.9

In [ ]:
#nested list to np array (np array is better for use later on)
#another 2 gb of ram
nparraylist_train = np.array(listylist)
# ---
nparraylist_testN0 = np.array(listylist_testN0)
nparraylist_testN1 = np.array(listylist_testN1)
nparraylist_testN2 = np.array(listylist_testN2)
nparraylist_testN3 = np.array(listylist_testN3)
#prior to this block: 5.9 gb
#after: 8.2

In [ ]:
#reshape parameters (3dim array to 2dim array)

lenDim1_train = len(nparraylist_train) #67625
lenDim1_testN0 = len(nparraylist_testN0) #684

lenDim2_train = len(nparraylist_train[0]) * len(nparraylist_train[0][0]) #5000
lenDim2_testN0 = len(nparraylist_testN0[0]) * len(nparraylist_testN0[0][0]) #5000

In [ ]:
#however these are 3 dims, and need to be 2.. (1 rows, 2nd the array of vectors..)
# this can be done with reshape (see below)

X_train_GloVe = np.reshape(
               nparraylist_train,     # the array to be reshaped
               (lenDim1_train, lenDim2_train)  # dimensions of the new array
              )

# --- (change to for loop to get these 4 sets reshaped in one block)
X_testN0_GloVe = np.reshape(
               nparraylist_testN0,     # the array to be reshaped
               (lenDim1_testN0, lenDim2_testN0)  # dims of test
              )

X_testN1_GloVe = np.reshape(
               nparraylist_testN1,     # the array to be reshaped
               (lenDim1_testN0, lenDim2_testN0)  # no diff between N0 and N1 ..
              )

X_testN2_GloVe = np.reshape(
               nparraylist_testN2,     # the array to be reshaped
               (lenDim1_testN0, lenDim2_testN0)  
              )

X_testN3_GloVe = np.reshape(
               nparraylist_testN3,     # the array to be reshaped
               (lenDim1_testN0, lenDim2_testN0) 
              )
#prior to this block: 8.2 gb
#after: 8.2